In [2]:
import torch
from torch import nn, optim
import numpy as np
from matplotlib import pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from sklearn.datasets import make_swiss_roll
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pickle as pkl
import random
from sklearn.model_selection import train_test_split
import glob
import MDAnalysis as mda
import itertools
from skorch import NeuralNetClassifier

In [4]:

def construct_encoder(input_dim, encoding_dim, dimensions_list, activation):
    
    encoder_layers = [nn.Linear(input_dim, dimensions_list[0]), activation]
    decoder_layers = [nn.Linear(encoding_dim, dimensions_list[-1]), activation]

    for i in range(len(dimensions_list) - 1):
        encoder_layers.append(nn.Linear(dimensions_list[i], dimensions_list[i+1]))
        encoder_layers.append(activation)
        decoder_layers.append(nn.Linear(dimensions_list[::-1][i], dimensions_list[::-1][i+1]))
        decoder_layers.append(activation)
        
    encoder_layers.append(nn.Linear(dimensions_list[-1], encoding_dim))
    decoder_layers.append(nn.Linear(dimensions_list[0], input_dim))

    encoder = nn.Sequential(*encoder_layers)
    decoder = nn.Sequential(*decoder_layers)
    
    class Autoencoder(nn.Module):

        def __init__(self, encoder, decoder):
            super(Autoencoder, self).__init__()
            self.encode = encoder
            self.decode = decoder
            
        def forward(self, x):
            x = self.encode(x)
            x = self.decode(x)
            return x

    return Autoencoder(encoder=encoder, decoder=decoder)


A = construct_encoder(10, 3, [7, 6, 5], nn.ReLU())


In [7]:
model = NeuralNetClassifier(
    module = construct_encoder()
)

TypeError: construct_encoder() missing 4 required positional arguments: 'input_dim', 'encoding_dim', 'dimensions_list', and 'activation'

In [6]:
print(model)

<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=Autoencoder(
    (encode): Sequential(
      (0): Linear(in_features=10, out_features=7, bias=True)
      (1): ReLU()
      (2): Linear(in_features=7, out_features=6, bias=True)
      (3): ReLU()
      (4): Linear(in_features=6, out_features=5, bias=True)
      (5): ReLU()
      (6): Linear(in_features=5, out_features=3, bias=True)
    )
    (decode): Sequential(
      (0): Linear(in_features=3, out_features=5, bias=True)
      (1): ReLU()
      (2): Linear(in_features=5, out_features=6, bias=True)
      (3): ReLU()
      (4): Linear(in_features=6, out_features=7, bias=True)
      (5): ReLU()
      (6): Linear(in_features=7, out_features=10, bias=True)
    )
  ),
)


In [6]:

def train(encoder, loss_function, optimizer, n_epochs, batch_size, X):
    
    encoder.train()
    indices = [i for i in range(X.shape[0])]
    
    for epoch in range(1, n_epochs + 1):
        
        random.shuffle(indices)
        batches = [i for i in range(0, len(indices), batch_size)]

        for i in range(len(batches) - 1):
            
            batch_X = X[indices[batches[i]:batches[i+1]]]
            optimizer.zero_grad()
            output = encoder(batch_X)
            loss = loss_function(output, batch_X)
            loss.backward()
            optimizer.step()
        
        print(f"epoch {epoch} \t Loss: {loss.item():.4g}")
    

In [7]:
X = torch.Tensor(np.load("drors_for_all.npy"))
ids = X[:,-1]
X = X[:,[0,1,2,3]]

In [8]:
AE = construct_encoder(X.shape[1], 2, [8], nn.ReLU())
loss_fn = nn.L1Loss()
optimizer = optim.Adam(AE.parameters(), lr=1e-3)

train(AE, loss_fn, optimizer, 10, 32, X)


epoch 1 	 Loss: 0.6083810329437256
epoch 2 	 Loss: 0.550406277179718
epoch 3 	 Loss: 0.4399120807647705
epoch 4 	 Loss: 0.49019306898117065
epoch 5 	 Loss: 0.41201677918434143
epoch 6 	 Loss: 0.46115174889564514
epoch 7 	 Loss: 0.3997623324394226
epoch 8 	 Loss: 0.45789313316345215
epoch 9 	 Loss: 0.4028245806694031
epoch 10 	 Loss: 0.37134426832199097


In [9]:
with torch.no_grad():
    encoded = AE.encode(X)
    decoded = AE.decode(encoded)
    err = loss_fn(decoded, X).item()
    print(f"Error on test set: {err}")
    E = decoded - X
    enc = encoded.cpu().detach().numpy()
    dec = decoded.cpu().detach().numpy()

Error on test set: 0.4530966281890869


In [19]:

def construct_params(X):
    
    next = 2**np.ceil(np.log(X.shape[1])/np.log(2))
    to_try = [next/4, next/2, next, 2*next, 4*next]


construct_params(np.zeros((1, 16)))

16.0


In [17]:
print(pow(2,4))

16
